In [55]:
!pip install opencv-python tensorflow

In [43]:
import os
import numpy as np
import cv2
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

# Image dimensions
num_px = 64

# Function to extract features and labels for training
def extract_features_and_labels_from_folders(base_dir):
    X = []
    y = []
    for label, folder_name in enumerate(['negative', 'positive']):  # 0 = negative, 1 = positive
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.exists(folder_path):
            print(f"Directory does not exist: {folder_path}")
            continue
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)


In [47]:
# Function to extract features for testing
def extract_features(directory):
    X = []
    filenames = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
        img = img / 255.0  # Normalize
        X.append(img)
        filenames.append(img_name)
    return np.array(X), filenames


In [49]:
# Load training data
train_dir = './training'
train_X, train_Y = extract_features_and_labels_from_folders(train_dir)

# One-hot encode labels
train_Y_one_hot = to_categorical(train_Y, 2)

In [51]:
# Build the CNN model
model = Sequential([
    Input(shape=(64, 64, 3)),  # Input layer with shape
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])


In [23]:
import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=20,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")

Training the model...
Epoch 1/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 48s 47ms/step - accuracy: 0.9407 - loss: 0.1369
Epoch 2/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 61s 61ms/step - accuracy: 0.9899 - loss: 0.0306
Epoch 3/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.9937 - loss: 0.0211
Epoch 4/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 56ms/step - accuracy: 0.9932 - loss: 0.0227
Epoch 5/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 333s 332ms/step - accuracy: 0.9958 - loss: 0.0141
Epoch 6/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - accuracy: 0.9966 - loss: 0.0122
Epoch 7/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 52s 52ms/step - accuracy: 0.9963 - loss: 0.0118
Epoch 8/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 52s 52ms/step - accuracy: 0.9966 - loss: 0.0123
Epoch 9/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 58s 58ms/step - accuracy: 0.9974 - loss: 0.0086
Epoch 10/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 59s 59ms/step - accuracy: 0.9973 - loss: 0.0097
Epoch 11/20
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 61s 60ms/step - accuracy: 0.9981

In [29]:
# Load testing data
test_dir = './test'
test_X, test_filenames = extract_features(test_dir)

# Predict on test data
print("Predicting on test data...")
predicted_classes = np.argmax(model.predict(test_X), axis=1)

# Map predictions to class names
class_map = {0: 'Negative', 1: 'Positive'}
predicted_labels = [class_map[pred] for pred in predicted_classes]

# Save results to CSV
output = pd.DataFrame({'Filename': test_filenames, 'Class': predicted_labels})
output.to_csv('test_results.csv', index=False)
print("Results saved to 'test_results.csv'")

Failed to load: ./test/.DS_Store
Predicting on test data...
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
Results saved to 'test_results.csv'


In [37]:
import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")

Training the model...
Epoch 1/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - accuracy: 0.9100 - loss: 0.1941
Epoch 2/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - accuracy: 0.9872 - loss: 0.0459
Epoch 3/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 54ms/step - accuracy: 0.9893 - loss: 0.0349
Epoch 4/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 55ms/step - accuracy: 0.9934 - loss: 0.0237
Epoch 5/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 60s 60ms/step - accuracy: 0.9947 - loss: 0.0168
Epoch 6/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 54ms/step - accuracy: 0.9956 - loss: 0.0145
Epoch 7/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.9959 - loss: 0.0133
Epoch 8/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 58s 57ms/step - accuracy: 0.9974 - loss: 0.0099
Epoch 9/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 56ms/step - accuracy: 0.9978 - loss: 0.0080
Epoch 10/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.9975 - loss: 0.0067
Total Training Time: 560.16397405 seconds
Final Training Accuracy: 99.7570633

In [53]:
import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=64,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")

Training the model...
Epoch 1/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 49s 96ms/step - accuracy: 0.8935 - loss: 0.2071
Epoch 2/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.9865 - loss: 0.0456
Epoch 3/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.9907 - loss: 0.0307
Epoch 4/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.9927 - loss: 0.0232
Epoch 5/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.9946 - loss: 0.0163
Epoch 6/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.9971 - loss: 0.0088
Epoch 7/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 52s 104ms/step - accuracy: 0.9970 - loss: 0.0100
Epoch 8/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 53s 105ms/step - accuracy: 0.9977 - loss: 0.0069
Epoch 9/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 55s 109ms/step - accuracy: 0.9981 - loss: 0.0062
Epoch 10/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 54s 107ms/step - accuracy: 0.9970 - loss: 0.0103
Total Training Time: 521.71095586 seconds
Final Training Accuracy: 99.72591400%


In [57]:
import os
import numpy as np
import cv2
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

# Image dimensions
num_px = 128

# Function to extract features and labels for training
def extract_features_and_labels_from_folders(base_dir):
    X = []
    y = []
    for label, folder_name in enumerate(['negative', 'positive']):  # 0 = negative, 1 = positive
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.exists(folder_path):
            print(f"Directory does not exist: {folder_path}")
            continue
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Function to extract features for testing
def extract_features(directory):
    X = []
    filenames = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
        img = img / 255.0  # Normalize
        X.append(img)
        filenames.append(img_name)
    return np.array(X), filenames
# Load training data
train_dir = './training'
train_X, train_Y = extract_features_and_labels_from_folders(train_dir)

# One-hot encode labels
train_Y_one_hot = to_categorical(train_Y, 2)

# Build the CNN model
model = Sequential([
    Input(shape=(128, 128, 3)),  # Input layer with shape
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])

import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")


Training the model...
Epoch 1/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 209s 207ms/step - accuracy: 0.9291 - loss: 0.1874
Epoch 2/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 204s 204ms/step - accuracy: 0.9905 - loss: 0.0287
Epoch 3/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 205s 204ms/step - accuracy: 0.9929 - loss: 0.0257
Epoch 4/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 205s 204ms/step - accuracy: 0.9918 - loss: 0.0261
Epoch 5/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 207s 207ms/step - accuracy: 0.9944 - loss: 0.0167
Epoch 6/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 205s 204ms/step - accuracy: 0.9957 - loss: 0.0147
Epoch 7/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 205s 204ms/step - accuracy: 0.9972 - loss: 0.0091
Epoch 8/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 220s 219ms/step - accuracy: 0.9967 - loss: 0.0119
Epoch 9/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 212s 211ms/step - accuracy: 0.9981 - loss: 0.0084
Epoch 10/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 205s 204ms/step - accuracy: 0.9970 - loss: 0.0097
Total Training Time: 2183.82775497 seconds
Final Training

In [59]:
import os
import numpy as np
import cv2
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

# Image dimensions
num_px = 64

# Function to extract features and labels for training
def extract_features_and_labels_from_folders(base_dir):
    X = []
    y = []
    for label, folder_name in enumerate(['negative', 'positive']):  # 0 = negative, 1 = positive
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.exists(folder_path):
            print(f"Directory does not exist: {folder_path}")
            continue
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Function to extract features for testing
def extract_features(directory):
    X = []
    filenames = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
        img = img / 255.0  # Normalize
        X.append(img)
        filenames.append(img_name)
    return np.array(X), filenames
# Load training data
train_dir = './training'
train_X, train_Y = extract_features_and_labels_from_folders(train_dir)

# One-hot encode labels
train_Y_one_hot = to_categorical(train_Y, 2)

# Build the CNN model
model = Sequential([
    Input(shape=(64, 64, 3)),  # Input layer with shape
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])

    
    
    

import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")


Training the model...
Epoch 1/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 68s 67ms/step - accuracy: 0.9210 - loss: 0.1737
Epoch 2/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 72s 71ms/step - accuracy: 0.9911 - loss: 0.0282
Epoch 3/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 70s 70ms/step - accuracy: 0.9947 - loss: 0.0183
Epoch 4/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 71s 71ms/step - accuracy: 0.9963 - loss: 0.0133
Epoch 5/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 72s 71ms/step - accuracy: 0.9973 - loss: 0.0111
Epoch 6/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 72s 72ms/step - accuracy: 0.9969 - loss: 0.0107
Epoch 7/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 72s 71ms/step - accuracy: 0.9979 - loss: 0.0060
Epoch 8/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 73s 72ms/step - accuracy: 0.9977 - loss: 0.0079
Epoch 9/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 73s 72ms/step - accuracy: 0.9981 - loss: 0.0059
Epoch 10/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 74s 74ms/step - accuracy: 0.9981 - loss: 0.0082
Total Training Time: 725.86887193 seconds
Final Training Accuracy: 99.8286962

In [61]:
import os
import numpy as np
import cv2
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

# Image dimensions
num_px = 64

# Function to extract features and labels for training
def extract_features_and_labels_from_folders(base_dir):
    X = []
    y = []
    for label, folder_name in enumerate(['negative', 'positive']):  # 0 = negative, 1 = positive
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.exists(folder_path):
            print(f"Directory does not exist: {folder_path}")
            continue
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Function to extract features for testing
def extract_features(directory):
    X = []
    filenames = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
        img = img / 255.0  # Normalize
        X.append(img)
        filenames.append(img_name)
    return np.array(X), filenames
# Load training data
train_dir = './training'
train_X, train_Y = extract_features_and_labels_from_folders(train_dir)

# One-hot encode labels
train_Y_one_hot = to_categorical(train_Y, 2)

# Build the CNN model
model = Sequential([
    Input(shape=(64, 64, 3)),  # Input layer with shape
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])

    
    
    

import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")


Training the model...
Epoch 1/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.8803 - loss: 0.2706
Epoch 2/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 41ms/step - accuracy: 0.9801 - loss: 0.0695
Epoch 3/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 44s 43ms/step - accuracy: 0.9885 - loss: 0.0446
Epoch 4/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 41ms/step - accuracy: 0.9892 - loss: 0.0362
Epoch 5/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 41ms/step - accuracy: 0.9887 - loss: 0.0363
Epoch 6/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.9926 - loss: 0.0232
Epoch 7/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.9930 - loss: 0.0216
Epoch 8/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - accuracy: 0.9931 - loss: 0.0202
Epoch 9/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - accuracy: 0.9955 - loss: 0.0142
Epoch 10/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.9956 - loss: 0.0135
Total Training Time: 425.95752811 seconds
Final Training Accuracy: 99.5452702

In [63]:
import os
import numpy as np
import cv2
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import to_categorical

# Image dimensions
num_px = 64

# Function to extract features and labels for training
def extract_features_and_labels_from_folders(base_dir):
    X = []
    y = []
    for label, folder_name in enumerate(['negative', 'positive']):  # 0 = negative, 1 = positive
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.exists(folder_path):
            print(f"Directory does not exist: {folder_path}")
            continue
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Function to extract features for testing
def extract_features(directory):
    X = []
    filenames = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load: {img_path}")
            continue
        img = cv2.resize(img, (num_px, num_px))  # Resize to 64x64
        img = img / 255.0  # Normalize
        X.append(img)
        filenames.append(img_name)
    return np.array(X), filenames
# Load training data
train_dir = './training'
train_X, train_Y = extract_features_and_labels_from_folders(train_dir)

# One-hot encode labels
train_Y_one_hot = to_categorical(train_Y, 2)

# Build the CNN model
model = Sequential([
    Input(shape=(64, 64, 3)),  # Input layer with shape
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification
])

    
    
    

import time
# Start timer
start_time = time.time()


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Training the model...")
history = model.fit(
    train_X,
    train_Y_one_hot,
    batch_size=32,
    epochs=10,
    verbose=1
)

# End timer
end_time = time.time()
training_time = end_time - start_time

# Get the final accuracy from the training history
final_accuracy = history.history['accuracy'][-1]

# Display results
print(f"Total Training Time: {training_time:.8f} seconds")
print(f"Final Training Accuracy: {final_accuracy:.8%}")


Training the model...
Epoch 1/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - accuracy: 0.9000 - loss: 0.2094
Epoch 2/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 53s 53ms/step - accuracy: 0.9843 - loss: 0.0494
Epoch 3/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 54s 54ms/step - accuracy: 0.9918 - loss: 0.0272
Epoch 4/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 54ms/step - accuracy: 0.9945 - loss: 0.0179
Epoch 5/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.9955 - loss: 0.0132
Epoch 6/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 56ms/step - accuracy: 0.9963 - loss: 0.0114
Epoch 7/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 55ms/step - accuracy: 0.9963 - loss: 0.0105
Epoch 8/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.9957 - loss: 0.0127
Epoch 9/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 55s 55ms/step - accuracy: 0.9980 - loss: 0.0067
Epoch 10/10
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.9984 - loss: 0.0059
Total Training Time: 553.39954591 seconds
Final Training Accuracy: 99.8380422